Import data

In [1]:
import requests
import csv

In [2]:
URL = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv'
response = requests.get(URL)
response.raise_for_status()

columns = ['track_id', 'track_name', 'track_artist', 'playlist_genre', 'playlist_subgenre', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']

data = response.text.splitlines()
data = csv.DictReader(data)
data = [{k: v for k, v in row.items() if k in columns} for row in data]
data[:5]

[{'track_id': '6f807x0ima9a1j3VPbc7VN',
  'track_name': "I Don't Care (with Justin Bieber) - Loud Luxury Remix",
  'track_artist': 'Ed Sheeran',
  'playlist_genre': 'pop',
  'playlist_subgenre': 'dance pop',
  'danceability': '0.748',
  'energy': '0.916',
  'key': '6',
  'loudness': '-2.634',
  'mode': '1',
  'speechiness': '0.0583',
  'acousticness': '0.102',
  'instrumentalness': '0',
  'liveness': '0.0653',
  'valence': '0.518',
  'tempo': '122.036',
  'duration_ms': '194754'},
 {'track_id': '0r7CVbZTWZgbTCYdfa2P31',
  'track_name': 'Memories - Dillon Francis Remix',
  'track_artist': 'Maroon 5',
  'playlist_genre': 'pop',
  'playlist_subgenre': 'dance pop',
  'danceability': '0.726',
  'energy': '0.815',
  'key': '11',
  'loudness': '-4.969',
  'mode': '1',
  'speechiness': '0.0373',
  'acousticness': '0.0724',
  'instrumentalness': '0.00421',
  'liveness': '0.357',
  'valence': '0.693',
  'tempo': '99.972',
  'duration_ms': '162600'},
 {'track_id': '1z1Hg7Vb0AhHDiEmnDE79l',
  'tra

Data Preprocessing

In [3]:
# create a one hot encoding for the playlist_genre and playlist_subgenre columns
playlist_genre = set(row['playlist_genre'] for row in data)
playlist_subgenre = set(row['playlist_subgenre'] for row in data)

for row in data:
    row['label'] = {}

for genre in playlist_genre:
    for subgenre in playlist_subgenre:
        column_name = f'{genre}_{subgenre}'
        for row in data:
            row['label'][column_name] = int(row['playlist_genre'] == genre and row['playlist_subgenre'] == subgenre)

# remove the original playlist_genre and playlist_subgenre columns
for row in data:
    del row['playlist_genre']
    del row['playlist_subgenre']

In [4]:
# create a one hot encoding for the key and mode columns
key = set(row['key'] for row in data)
mode = set(row['mode'] for row in data)

for k in key:
    for m in mode:
        column_name = f'keymode_{k}_{m}'
        for row in data:
            row[column_name] = int(row['key'] == k and row['mode'] == m)

# remove the original key and mode columns
for row in data:
    del row['key']
    del row['mode']

Feature engineering: normalization of continuous properties of songs

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import numpy as np

for track in data:
    track['danceability'] = float(track['danceability'])
    track['energy'] = float(track['energy'])
    track['loudness'] = float(track['loudness'])
    track['speechiness'] = float(track['speechiness'])
    track['acousticness'] = float(track['acousticness'])
    track['instrumentalness'] = float(track['instrumentalness'])
    track['liveness'] = float(track['liveness'])
    track['valence'] = float(track['valence'])
    track['tempo'] = float(track['tempo'])
    track['duration_ms'] = int(track['duration_ms'])

# list of continuous features in our dataset
continuous_features = [
    'danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms'
]

features_df = pd.DataFrame([{
    feat: track[feat] for feat in continuous_features
} for track in data])

# apply a standard scalar for normalization
scaler = StandardScaler() 
scaled_features = scaler.fit_transform(features_df)

# replacing data with normalization applied
for idx, track in enumerate(data):
    for i, feat in enumerate(continuous_features):
        track[feat] = scaled_features[idx][i]

# for track in data:
#     track['duration_ms'] = np.log1p(track['duration_ms'])  # log(x+1) to avoid log(0)
#     track['loudness'] = np.log1p(abs(track['loudness'])) * np.sign(track['loudness'])

In [ ]:
import csv

# Function to flatten the dictionary
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):  # Recursively flatten nested dictionaries
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Flatten each dictionary in the list
flattened_data = []

for item in data:
    flattened_item = flatten_dict(item)
    flattened_data.append(flattened_item)

print(type(flattened_data[0]))

# # Get the fieldnames from the keys of the first flattened dictionary
# fieldnames = flattened_data[0].keys()

# # Write the flattened data to CSV
# with open('tracks.csv', 'w', newline='', encoding='utf-8') as csvfile:
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()  # Write the header
#     writer.writerows(flattened_data)  # Write the data

# print("Data written to 'tracks.csv'")

[]
